In [1]:
%load_ext autoreload
%autoreload 1
%matplotlib inline
%aimport classifier
# Imports

import numpy as np
import sys
import os
from skimage import io
from datetime import datetime

import matplotlib
from matplotlib import pyplot as plt


import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets


# Constants
classes = [
     'bighorn-sheep',
     'bison',
     'black-stork',
     'brown-bear',
     'bullfrog',
     'camel',
     'gazelle',
     'golden-retriever',
     'goldfish',
     'ladybug',
     'lion',
     'orangutan',
     'penguin',
     'persian-cat',
     'pig',
     'puma'
]


ModuleNotFoundError: No module named 'torch'

In [1]:
def show_image(data):
    '''
    Given an image-like array, plot it as an image.
    
    Parameters:
        data: a numpy array shaped like an image (could be RGB or grayscale)
    '''
    plt.imshow(data, interpolation='nearest')
    plt.show()

In [ ]:
def get_N_images(N, class_name, root_folder):
    """
    Returns N images from [class_string] class under [root_folder]
    
    Inputs:
        N             scalar; number of images to return
        
        class_name    string; name of class to get images from.
                      Must be name of a folder under root_folder
                      where images of the class are held.
                      
        root_folder   string; relative path to folder containing subfolders
                      with image classes.
                      
                      Example structure:
                      root_folder/
                          class_x/
                              XXX.JPEG
    Returns:
        images        length N list of numpy arrays of images of class [class_name]
    """
    images = []
    for root, dir, files in os.walk(root_folder):
        if root.split(os.sep)[-1] == class_name:
            for i, file in enumerate(files):
                if file.endswith('.JPEG'):
                    images.append(plt.imread(os.path.join(root,file)))
                if i+1 == N:
                    return images


In [ ]:
N = 5
class_name = 'pig'
folder     = os.path.join('.', 'data', 'train')
images     = get_N_images(N, class_name, folder)

print("Visualizing class: {}".format(class_name))
for i in range(N):
    show_image(images[i])

In [ ]:
dataset_means = [123./255., 116./255.,  97./255.]
dataset_stds  = [ 54./255.,  53./255.,  52./255.]


transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.ToTensor(),
            transforms.Normalize(dataset_means, dataset_stds)
        ])

In [ ]:
class AnimalDataset(Dataset):
    def __init__(self, root_dir, classes, transform=None):
        """
        Args:
            root_dir (string):
                Directory with all the images.
                Of the form:
                root_dir/
                    <class_XX>/
                        <XXXX>.JPEG
                        ...
                    <class_XX>/
                        <XXXX>.JPEG
                        ...
                where <class_XX> is replaced with the class name, and <XXXX>.JPEG
                are the images. Must be .JPEG extension.
                
            classes (list of strings): list of class names, same as names of
                subfolders under root_dir
                
            transform (callable, optional): Optional transform to be applied
                on a sample.
                
        """
        self.root_dir = root_dir
        self.classes = classes
        self.transform = transform

        self.raw_data = []
        for i, cl in enumerate(self.classes):
            for root, directory, files in os.walk(os.path.join(root_dir, cl)):
                for file in files:
                    if '.JPEG' in file:
                        self.raw_data.append((os.path.join(root, file), i))

    def __len__(self):
        return len(self.raw_data)

    def __getitem__(self, idx):
        if type(idx) == torch.Tensor:
            idx = idx.item()
        
        image_path, label = (self.raw_data[idx])
        image = io.imread(image_path)
        image = self.transform(image)
        label = torch.tensor([label], dtype=torch.long)
        return image, label

In [ ]:
BATCH_SIZE = 128

train_dataset = AnimalDataset(os.path.join('.', 'data', 'train'),classes, transform=transform)
val_dataset   = AnimalDataset(os.path.join('.', 'data', 'val'),  classes, transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
# %aimport classifier

net_pretrained = classifier.AnimalBaselineNet()

# Load pretrained weights into network to check if architecture is correct
weights_path = os.path.join('.', 'models', 'baseline.pth')
net_pretrained.load_state_dict(torch.load(weights_path, map_location="cpu"))

for layer in net_pretrained.children():
    print(layer)


In [ ]:
%aimport classifier

net = classifier.AnimalBaselineNet()

criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(net.parameters(), lr = 0.01)
epochs = 30

In [ ]:
def log_progress(curr_batch, batch_size, curr_epoch, total_epochs, dataloader):
    """
    Prints message logging progress through training.
    """
    progress = float(curr_batch + 1)/(float(len(dataloader.dataset)) / batch_size)
    log = "EPOCH [{}/{}].Progress: {} % ".format(
        curr_epoch + 1, total_epochs, round(progress * 100, 2))
    sys.stdout.write("\r" + log)
    sys.stdout.flush()

In [ ]:
%aimport classifier

# Keep track of average losses, training accuracy and validation accuracy for each epoch
train_loss_history = np.zeros(epochs)
train_acc_history  = np.zeros(epochs)
val_loss_history   = np.zeros(epochs)
val_acc_history    = np.zeros(epochs)

start_time = datetime.now()

for epoch in range(epochs):
    
     # ============================ Training ==============================
    print("Training...")
    
    # Place network in training mode
    net.train()
    
    # Initialize running epoch loss and number correctly classified
    running_loss   = 0.0
    num_correct    = 0.0
    total_images   = 0.0
    
    
    for batch_num, (inputs, labels) in enumerate(train_dataloader):
        # [inputs] and [labels] is one batch of images and their classes

        log_progress(batch_num, BATCH_SIZE, epoch, epochs, train_dataloader)
        
        # Function call to classifier
        curr_loss, curr_correct, curr_images = \
            classifier.model_train(net, inputs, labels, criterion, optimizer)
        running_loss += curr_loss
        num_correct += curr_correct
        total_images += curr_images

    # Update statistics for epoch
    train_loss_history[epoch] = running_loss / total_images
    train_acc_history[epoch]  = float(num_correct)  / float(total_images)
    print("\n Train Avg. Loss: [{}] Acc: {} on {} images\n".format(
          round(train_loss_history[epoch],4), train_acc_history[epoch], total_images) )
    
    # ============================ Validation ==============================
    print("Validating...")
    # Place network in testing mode (won't need to keep track of gradients)
    net.eval()
    
    running_loss   = 0.0
    num_correct    = 0.0
    total_images   = 0.0
    
    for batch_num, (inputs, labels) in enumerate(val_dataloader):
        
        # Propagate batch through network
        outputs  = net(inputs)
                                                 
        # Calculate loss
        loss     = criterion(outputs, labels.squeeze())
                                                 
        # Prediction is class with highest class score
        _, preds = torch.max(outputs, 1)
        
        running_loss  += loss.item()
        num_correct   += torch.sum(preds == labels.data.reshape(-1))
        total_images  += labels.data.numpy().size
        
    # Update statistics for validation data
    val_loss_history[epoch] = running_loss / total_images
    val_acc_history[epoch]  = float(num_correct)  / float(total_images) 
    print("Val Avg. Loss: [{}] Acc: {} on {} images\n".format(
        round(val_loss_history[epoch],4), val_acc_history[epoch], total_images))
    
print("Time Elapsed: {} seconds".format(
    (datetime.now() - start_time).total_seconds()))

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(11,5))

ax = axs[0]
ax.set_title("Average Losses", fontsize=20)
ax.plot(train_loss_history, label="Training")
ax.plot(val_loss_history,   label="Validation")
ax.set_xlabel("Epoch",        fontsize=16)
ax.set_ylabel("Average loss", fontsize=16)
ax.legend(loc="best",         fontsize=16)

ax = axs[1]
ax.set_title("Accuracy", fontsize=22)
ax.plot(train_acc_history, label="Training")
ax.plot(val_acc_history,   label="Validation")
ax.set_xlabel("Epoch",        fontsize=16)
ax.set_ylabel("Accuracy",     fontsize=16)
ax.legend(loc="best",         fontsize=16)

In [ ]:
def get_dataloader(transform):
    """
    Returns dataloader for AnimalDataset given input transform, with batch size 1.
    """
    dataset = AnimalDataset(os.path.join('.', 'data', 'val'),  classes, transform=transform)
    return DataLoader(dataset, batch_size=1, shuffle=False)

In [ ]:
%aimport classifier

# Define display function to convert image to correct format
to_pil = transforms.ToPILImage()
convert = lambda image: to_pil(image.squeeze())

fig, axs = plt.subplots(2, 3, figsize=(15,10))

ax = axs[0,0]
ax.set_title("Original")
transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.ToTensor()
        ])
img, label = next(iter(get_dataloader(transform)))
ax.imshow(convert(img))

ax = axs[0,1]
ax.set_title("Constrast")
transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.ToTensor(),
            classifier.Contrast(min_contrast=0.3, max_contrast=0.9)
        ])
img, label = next(iter(get_dataloader(transform)))
ax.imshow(convert(img))

ax = axs[0,2]
ax.set_title("Shift")
transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.ToTensor(),
            classifier.Shift(max_shift=5)
        ])
img, label = next(iter(get_dataloader(transform)))
ax.imshow(convert(img))

ax = axs[1,0]
ax.set_title("Rotate")
transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.ToTensor(),
            classifier.Rotate(max_angle=10)
        ])
img, label = next(iter(get_dataloader(transform)))
ax.imshow(convert(img))

ax = axs[1,1]
ax.set_title("Flip")
transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.ToTensor(),
            classifier.HorizontalFlip(p=1)
        ])
img, label = next(iter(get_dataloader(transform)))
ax.imshow(convert(img))


ax = axs[1,2]
ax.set_title("Combination")
transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.ToTensor(),
            classifier.Contrast(min_contrast=0.3, max_contrast=0.9),
            classifier.Shift(max_shift=5),
            classifier.Rotate(max_angle=10),
            classifier.HorizontalFlip(p=0.5),
        ])
img, label = next(iter(get_dataloader(transform)))
ax.imshow(convert(img))


In [ ]:
%aimport classifier

# Load pretrained baseline model
net          = classifier.AnimalBaselineNet()
weights_path = os.path.join('.', 'models', 'baseline.pth')
net.load_state_dict(torch.load(weights_path, map_location="cpu"))

# Specify parameters for forward pass
# Don't normalize images for purpose of visualization
transform = transforms.Compose([
            transforms.ToTensor()
        ])
val_dataset    = AnimalDataset(os.path.join('.', 'data', 'val'),  classes, transform=transform)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=True)

criterion = nn.CrossEntropyLoss()
epsilon = 0.02

original_imgs      = []
original_preds     = []

adversarial_imgs   = []
adversarial_preds  = []
adversarial_noises = []


for i, (img, label) in enumerate(val_dataloader):
    
    # Set image tensor so gradient is calculated
    img.requires_grad = True
    
    output = net(img)

    init_pred = output.max(1, keepdim=True)[1]
    
    if init_pred == label:
        # Image classified correctly; generate adversarial image
        perturbed_img, noise = classifier.get_adversarial(img, output, label[0], net, criterion, epsilon)
    
        adversarial_output = net(perturbed_img)
        adversarial_pred   = adversarial_output.max(1, keepdim=True)[1]

        original_imgs.append(to_pil(img.squeeze()))
        original_preds.append(init_pred)
        adversarial_imgs.append(to_pil(perturbed_img.squeeze()))
        adversarial_preds.append(adversarial_pred)
        adversarial_noises.append(to_pil(noise.squeeze()))
    


print("Out of total {} images generated, {} % of adversarial images misclassified".format(
    len(original_imgs),
    round((torch.sum(torch.Tensor(original_preds) != torch.Tensor(adversarial_preds)).item())
        / len(original_imgs) * 100., 4)))



In [ ]:
num_desired_adversarial = 4

fig, axs = plt.subplots(num_desired_adversarial, 3, figsize=(10,3*num_desired_adversarial))
matplotlib.rcParams.update({'font.size': 14})


for i in range(num_desired_adversarial):
    ax = axs[i,0]
    ax.set_title('Original: {}'.format(classes[original_preds[i]]))
    ax.imshow(original_imgs[i])
    
    ax = axs[i,1]
    ax.set_title('Adversarial Noise')
    ax.imshow(adversarial_noises[i])
    
    ax = axs[i,2]
    ax.set_title('Adversarial: {}'.format(classes[adversarial_preds[i]]))
    ax.imshow(adversarial_imgs[i])
    
plt.tight_layout()
plt.show()
    